# 1. Import Dependencies

In [1]:
!pip install opencv-python

In [2]:
# Import opencv
import cv2

# Import uuid
import uuid

# Import Operating System
import os

# Import time
import time

#Import Numpy
import numpy as np

import shutil

import random as rd

# 2. Define Images to Collect

In [3]:
#Specify What You Want Labelled As Well As Empty Train & Test Folders For Later
labels = ['Mask', 'NoMask']
number_imgs = 5

# 3. Setup Folders 

In [4]:
#PATHS
FOLDER_PATH = os.getcwd()
TF_PATH = os.path.join(FOLDER_PATH,'Tensorflow','workspace','images', 'collectedimages')
IMG_PATH = os.path.join(FOLDER_PATH, 'Tensorflow','workspace','images')

DOWNLOAD_IMG_PATH = os.path.join(FOLDER_PATH, 'Images')

folders = ["Train", "Test"]

In [ ]:
TF_PATH

In [5]:
#Create Folders for Each Label
if not os.path.exists(TF_PATH):
    if os.name == 'posix':
        !mkdir -p {TF_PATH}
    if os.name == 'nt':
        !mkdir {TF_PATH}

#Create Train & Test Folders
os.chdir(IMG_PATH)
for folder in folders:
    path = os.path.join(IMG_PATH, folder)
    if not os.path.exists(path):
        !mkdir {path}
        print("Folder Created For {}....".format(folder))
    else:
        print("Folder Already Created For {}....".format(folder))
        
#Create Folders For Custom Labels
for label in labels:
    path = os.path.join(TF_PATH, label)
    if not os.path.exists(path):
        !mkdir {path}
        print("Folder Created For {}....".format(label))
    else:
        print("Folder Already Created For {}....".format(label))

Folder Already Created For Train....
Folder Already Created For Test....
Folder Already Created For Mask....
Folder Already Created For NoMask....


# 4. Capture Images On Camera (Optional)

In [ ]:
save_Files_PATH = os.path.join(IMG_PATH,'collectedimages')

for label in labels:
    cap = cv2.VideoCapture(0)
    print('Collecting images for {}'.format(label))
    time.sleep(5)
    for imgnum in range(number_imgs):
        print('Collecting image {}'.format(imgnum))
        ret, frame = cap.read()
        imgname = os.path.join(save_Files_PATH,label,label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname, frame)
        cv2.imshow('frame', frame)
        time.sleep(2)

        if cv2.waitKey(1) == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

# 4. Copy Images From Other Folder

In [ ]:
for label in labels:
    src = os.path.join(DOWNLOAD_IMG_PATH, label)
    dest = os.path.join(TF_PATH, label)
    os.chdir(src)
    files = os.listdir(src)
    if len(os.listdir(dest))== 0:
        for file in files:
            shutil.copy(file, dest)
        print("Files Copied To Destination Folder....")
    else:
        print("Files Already Copied From {} Folder...".format(label))

# 5. Renaming Images

In [ ]:
#Rename Images to <label>.num.jpg

for label in labels:
    DST_label_PATH = os.path.join(TF_PATH, label)
    os.chdir(DST_label_PATH)
    DIR = os.listdir(DST_label_PATH)
    for count, file_name in enumerate(DIR):
        src = DST_label_PATH + '\\' + file_name
        dst = DST_label_PATH + '\\' + label + "_" + str(count+1) + ".jpg"
        os.rename(src, dst)
print("Images Successfully Renamed...")

# 6. Import Image Labelling 

In [ ]:
!pip install --upgrade pyqt5 lxml

In [ ]:
LABEL_IMG_PATH = os.path.join(FOLDER_PATH,'Tensorflow','labelImg')

In [ ]:
#Create a Folder For Image Labeling 
if not os.path.exists(LABEL_IMG_PATH):
    !mkdir {LABEL_IMG_PATH}
    !git clone https://github.com/tzutalin/labelImg {LABEL_IMG_PATH}
    print("Folder Created and Repo Copied...")
else:
    print("Folder Already Created and Repo Copied....")

In [ ]:
if os.name == 'posix':
    !cd {LABEL_IMG_PATH} && make qt5py3
if os.name =='nt':
    !cd {LABEL_IMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc

In [ ]:
#Open Label Image Program
!cd {LABEL_IMG_PATH} && python labelImg.py

# 7. Splitting Images Into Ttraining & Testing

In [17]:
#Make a function to move files to Train folder

def move_file_Train(src_path):
    dst = os.path.join(IMG_PATH, 'Train')
    src = src_path
    shutil.move(src, dst)
    print("File moved to {}".format(dst))

In [18]:
#Make a function to move files to Test folder

def move_file_Test(src_path):
    dst = os.path.join(IMG_PATH, 'Test')
    src = src_path
    shutil.move(src, dst)
    print("File moved to {}".format(dst))

In [23]:
def rand_num_gen(length_dir):
    randomNum = rd.randrange(1, length_dir)
    return randomNum

In [ ]:
#Training Ratio Split Into Train Folder
TRAIN_RATIO = 0.7
randList = []
for label in labels:
    DST_label_PATH = os.path.join(TF_PATH, label)
    os.chdir(DST_label_PATH)
    DIR = os.listdir(DST_label_PATH)
    NUM_IMG_SPLIT = len(DIR)*TRAIN_RATIO
    print("Number of Images For Training: {}".format(int(NUM_IMG_SPLIT)))
    for x in range(int(NUM_IMG_SPLIT)):
        randNum = rand_num_gen(len(DIR))
        if len(randList) != int(NUM_IMG_SPLIT) and randNum not in randList:
            randList.append(randNum)
        else:
            while (randNum in randList):
                randNum = rand_num_gen(len(DIR))
            if len(randList) != int(NUM_IMG_SPLIT) and randNum not in randList:
                randList.append(randNum)
        
        IMG_SELECT = os.path.join(DST_label_PATH, label + "_" + str(randList[x]) + ".jpg")
        print(IMG_SELECT)
        move_file_Train(IMG_SELECT)  


In [26]:
#Training Ratio Split Into Test Folder

for label in labels:
    DST_label_PATH = os.path.join(TF_PATH, label)
    os.chdir(DST_label_PATH)
    DIR = os.listdir(DST_label_PATH)
    NUM_IMG_SPLIT = len(DIR)
    print("Number of Images For Training: {}".format(int(NUM_IMG_SPLIT))) 
    for file_name in DIR:
        file = os.path.join(DST_label_PATH, file_name)
        move_file_Test(file)    

Number of Images For Training: 0
Number of Images For Training: 0
